In [1]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = bert_model(**inputs)
    # Obtain the embeddings by averaging the last hidden states
    embeddings = outputs.last_hidden_state.mean(1)
    return embeddings


c:\Users\ujesh\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
# import torch.nn as nn
# import torch.nn.functional as F

# class MultimodalCrossAttention(nn.Module):
#     def __init__(self, text_embedding_dim, image_channels, hidden_dim, num_heads):
#         super().__init__()
#         self.text_projection = nn.Linear(text_embedding_dim, hidden_dim)
#         self.image_projection = nn.Conv2d(image_channels, hidden_dim, kernel_size=1)  # Project to the same hidden_dim
#         self.attention = nn.MultiheadAttention(hidden_dim, num_heads)
        
#     def forward(self, text_embeddings, image_latents):
#         # Project text embeddings
#         text_features = self.text_projection(text_embeddings)  # Shape: [batch_size, hidden_dim]
#         text_features = text_features.permute(1, 0)  # Shape for attention: [seq_len, batch_size, hidden_dim]
        
#         # Project image features
#         batch_size, _, _ = image_latents.shape
#         image_features = self.image_projection(image_latents)  # Shape: [batch_size, hidden_dim, H, W]
#         image_features = image_features.flatten(2)  # Shape: [batch_size, hidden_dim, H*W]
#         image_features = image_features.permute(2, 0, 1)  # Shape for attention: [seq_len, batch_size, hidden_dim]
        
#         # Apply cross-attention
#         # Here, we use image features as queries and text features as keys and values
#         attn_output, _ = self.attention(query=image_features, key=text_features, value=text_features)
        
#         # Post-processing of attention output can be added here
        
#         return attn_output


In [31]:
# import numpy as np

# text = "Example text for processing"
# text_embeddings = get_bert_embeddings(text)  # [1, 768] for single sentence
# print(text_embeddings.shape)

# # Simulate a latent image tensor of shape [1, 3, 64, 64]
# latent_image = np.load("D:/FYP/latent_vector.npy")  # Example latent image tensor
# print(latent_image.shape)

# # Instantiate the multimodal model
# model = MultimodalCrossAttention(text_embedding_dim=768,  # Dimension of BERT-base embeddings
#                                     image_channels=3, hidden_dim=512, num_heads=8)

# # Ensure text embeddings and latent image are on the same device
# if torch.cuda.is_available():
#     model = model.cuda()
#     text_embeddings = text_embeddings.cuda()
#     latent_image = latent_image.cuda()

# # Get combined features
# combined_features = model(text_embeddings, latent_image)

# print(combined_features.shape) 

torch.Size([1, 768])
(1, 64, 64, 3)


ValueError: too many values to unpack (expected 3)

In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Assuming your inputs
text_embedding = get_bert_embeddings("hello im aadhith")  # [Batch, TextEmbeddingDim]
latent_image = np.load("D:/FYP/latent_vector.npy")  # [Batch, H, W, Channels]
latent_vector_tensor = torch.from_numpy(latent_image)

# Convert image to PyTorch's channel-first format and apply adaptive avg pooling
latent_vector_tensor = latent_vector_tensor.permute(0, 3, 1, 2)  # [Batch, Channels, H, W]
pooled_image = F.adaptive_avg_pool2d(latent_vector_tensor, (8, 8))  # Reduce spatial dimensions
image_features = pooled_image.flatten(start_dim=2)  # [Batch, Channels, NewH*NewW]
image_features = image_features.permute(0, 2, 1)  # Prepare for attention [Batch, SeqLen, Channels]

print(text_embedding.shape)
print(image_features.shape)


torch.Size([1, 768])
torch.Size([1, 64, 3])


In [93]:
query = text_embedding.unsqueeze(0)  # Add sequence length dimension
query.shape
key_value = image_features
key_value.shape

# Step 1: Define a linear projection layer for tensor2
projection_layer = nn.Linear(in_features=3, out_features=768)

# Step 2: Apply the linear projection to tensor2
# Reshape tensor2 to [64, 3] to apply linear projection, then reshape back to [1, 64, 768]
tensor2_projected = projection_layer(key_value.view(-1, 3)).view(1, 64, 768)

# Step 3: Expand tensor1 to match the sequence length of tensor2_projected
tensor1_expanded = query.expand(-1, 64, -1)  # Size: [1, 64, 768]

print(tensor1_expanded.shape)
print(tensor2_projected.shape)

query = tensor1_expanded
key_value = tensor2_projected
attention_scores = torch.bmm(tensor1_expanded, tensor2_projected.transpose(1, 2))
print(attention_scores.shape)
attention_weights = F.softmax(attention_scores, dim=-1)

# Compute the weighted sum of values
attention_output = torch.bmm(attention_weights, tensor2_projected)

print(attention_output)
print(attention_output.shape)

torch.Size([1, 64, 768])
torch.Size([1, 64, 768])
torch.Size([1, 64, 64])
tensor([[[-0.4287, -0.3994, -0.7315,  ...,  0.0175, -0.2980,  0.0425],
         [-0.4287, -0.3994, -0.7315,  ...,  0.0175, -0.2980,  0.0425],
         [-0.4287, -0.3994, -0.7315,  ...,  0.0175, -0.2980,  0.0425],
         ...,
         [-0.4287, -0.3994, -0.7315,  ...,  0.0175, -0.2980,  0.0425],
         [-0.4287, -0.3994, -0.7315,  ...,  0.0175, -0.2980,  0.0425],
         [-0.4287, -0.3994, -0.7315,  ...,  0.0175, -0.2980,  0.0425]]],
       grad_fn=<BmmBackward0>)
torch.Size([1, 64, 768])


In [87]:
# # tensor_a = torch.randn(1, 64, 768)  # Q
# # tensor_b = torch.randn(1, 64, 768)  # K and V
# # Calculate the dot product between Q and K
# # Compute the dot product along the last dimension

    
# tensor1 = query
# tensor2 = key_value

# print(tensor1.shape)
# print(tensor2.shape)
# result = []

# # Iterate over the middle dimension
# for i in range(64):
#     dot_product = 0
#     # Compute dot product for the i-th vector
#     for j in range(768):
#         try:
#             dot_product += tensor1[0][i][j] * tensor2[0][i][j]
#         except:
#             print(i,j)
#     result.append(dot_product)

# print(result)
# # attention_scores = torch.matmul(query, key_value.transpose(-2, -1))
# attention_scores = torch.tensor(result)
# print(attention_scores.size())
# # Scale scores by the square root of the dimension of the keys
# dk = key_value.size(-1)
# print(dk)
# scaled_attention_scores = attention_scores / torch.sqrt(dk)

# # Apply softmax to get the attention weights
# attention_weights = F.softmax(scaled_attention_scores, dim=-1)

# # Multiply by V
# output = torch.matmul(attention_weights, key_value)

torch.Size([1, 64, 768])
torch.Size([1, 64, 768])
[tensor(-3.5360, grad_fn=<AddBackward0>), tensor(-3.6139, grad_fn=<AddBackward0>), tensor(-3.5560, grad_fn=<AddBackward0>), tensor(-3.4886, grad_fn=<AddBackward0>), tensor(-3.4993, grad_fn=<AddBackward0>), tensor(-3.4754, grad_fn=<AddBackward0>), tensor(-3.4880, grad_fn=<AddBackward0>), tensor(-3.4744, grad_fn=<AddBackward0>), tensor(-3.5635, grad_fn=<AddBackward0>), tensor(-3.6198, grad_fn=<AddBackward0>), tensor(-3.5684, grad_fn=<AddBackward0>), tensor(-3.5324, grad_fn=<AddBackward0>), tensor(-3.5239, grad_fn=<AddBackward0>), tensor(-3.5065, grad_fn=<AddBackward0>), tensor(-3.5538, grad_fn=<AddBackward0>), tensor(-3.5725, grad_fn=<AddBackward0>), tensor(-3.4120, grad_fn=<AddBackward0>), tensor(-3.4404, grad_fn=<AddBackward0>), tensor(-3.3314, grad_fn=<AddBackward0>), tensor(-3.2938, grad_fn=<AddBackward0>), tensor(-3.2256, grad_fn=<AddBackward0>), tensor(-3.3041, grad_fn=<AddBackward0>), tensor(-3.5448, grad_fn=<AddBackward0>), tensor

TypeError: sqrt(): argument 'input' (position 1) must be Tensor, not int

In [94]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class TransformerEncoderModel(nn.Module):
    def __init__(self, feature_size, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerEncoderModel, self).__init__()
        self.pos_encoder = PositionalEncoding(feature_size, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model=feature_size, nhead=nhead, dim_feedforward=nhid, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer=encoder_layers, num_layers=nlayers)
        self.feature_size = feature_size

    def forward(self, src):
        src = src * math.sqrt(self.feature_size)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        return output

# Parameters for the Transformer model
feature_size = 768  # Feature size (embedding dimension)
nhead = 8  # Number of heads in the multiheadattention models
nhid = 768  # Dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 6  # Number of nn.TransformerEncoderLayer in nn.TransformerEncoder
dropout = 0.2  # Dropout value

model = TransformerEncoderModel(feature_size, nhead, nhid, nlayers, dropout)

# Assuming input tensor is of shape [batch size, sequence length, features]
input_tensor = attention_output

# Transpose the input to match the expected format [sequence length, batch size, features]
input_tensor_transposed = input_tensor.transpose(0, 1)  # Shape: [64, 1, 768]

output = model(input_tensor_transposed)
print(output.shape)  # Should be [64, 1, 768] (sequence length, batch size, features)


NameError: name 'TransformerEncoderLayer' is not defined

In [54]:
class CrossAttentionModule(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(CrossAttentionModule, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embed_size, num_heads=num_heads)
        
    def forward(self, query, key_value):
        # Cross-attention: Query from text, Key/Value from image
        attn_output, _ = self.attention(query=query, key=key_value, value=key_value)
        return attn_output

class AdjustKeyValueDimensions(nn.Module):
    def __init__(self, key_value_dim, target_embedding_dim):
        super().__init__()
        self.projection = nn.Linear(key_value_dim, target_embedding_dim)

    def forward(self, key_value):
        batch_size, seq_len, _ = key_value.shape
        key_value_flat = key_value.view(batch_size * seq_len, -1)  # Flatten to [batch_size*seq_len, key_value_dim]
        projected = self.projection(key_value_flat)  # Project to [batch_size*seq_len, target_embedding_dim]
        projected = projected.view(batch_size, seq_len, -1)  # Reshape back to [batch_size, seq_len, target_embedding_dim]
        return projected

# Initialize module
embed_size = 768  # Assuming text embedding size
num_heads = 8  # Choose based on your model architecture
cross_attention = CrossAttentionModule(embed_size, num_heads)

# Prepare inputs
# Text embedding as query: [SeqLen, Batch, EmbeddingDim]
query = text_embedding.unsqueeze(0)  # Add sequence length dimension
print(query.shape)
# Image features as key/value: [SeqLen, Batch, EmbeddingDim]
key_value = image_features
print(image_features.shape)


# Adjust the key/value tensor
adjuster = AdjustKeyValueDimensions(key_value_dim=3, target_embedding_dim=768)
adjusted_key_value_tensor = adjuster(key_value)

# Now both tensors are compatible for cross-attention
print("Adjusted Key/Value Shape:", adjusted_key_value_tensor.shape)


# Apply cross-attention
attn_output = cross_attention(query, adjusted_key_value_tensor)


torch.Size([1, 1, 768])
torch.Size([1, 64, 3])
Adjusted Key/Value Shape: torch.Size([1, 64, 768])


RuntimeError: shape '[1, 8, 96]' is invalid for input of size 49152